In [63]:
import os
import glob
import json
import pymworks
import re
import datautils
import copy
import math
import time

import multiprocessing as mp
import numpy as np
import pandas as pd
import seaborn as sns
import pylab as pl
import cPickle as pkl
from cPickle import PicklingError

from scipy import stats

In [3]:
%matplotlib inline

### Some standard formatting functions:

In [4]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]


In [5]:
def parse_datafile_name(dfn):
    fn = os.path.splitext(os.path.split(dfn)[-1])[0]
    fparts = fn.split('_')

    assert len(fparts) == 2, "*Warning* Unknown naming fmt: %s" % str(fparts)
    animalid = fparts[0]
    datestr = fparts[1]

    # Make sure no exra letters are in the datestr (for parta, b, etc.)
    if not datestr.isdigit():
        datestr = re.split(r'\D', datestr)[0] # cut off any letter suffix
    if len(datestr) == 6:
        session = '20%s' % datestr 
    elif len(datestr) == 8:
        session = datestr 

    return animalid, session

### File parsing functions:

In [6]:
def get_run_time(df):
    state_modes = df.get_events('#state_system_mode')
    run_bounds = None
    
    while True:
        try:
            running = next(d for d in state_modes if d.value==2)
            start_time = running.time
            strt = state_modes.index(running)
        except StopIteration:
            return run_bounds

        try:
            stopping = next(d for d in state_modes[strt:] if d.value != 2) 
            end_time = stopping.time
            stp = state_modes.index(stopping)
        except StopIteration:
            end_time = df.get_maximum_time()
            stp = 0

        if run_bounds is None:
            run_bounds = []
        
        run_bounds.append((start_time, end_time))

        # Check if there are additional run chunks:
        remaining_state_evs = state_modes[stp:]
        additional_starts = [s for s in remaining_state_evs if s.value == 2]
        if len(additional_starts) > 0:
            state_modes = remaining_state_evs
        else:
            break
        

    return run_bounds

In [7]:
# blacklisttests = [
#     #lambda s: (('name' in s.keys()) and (s['name'] == 'pixel clock')),
#     lambda s: (('type' in s.keys()) and (s['type'] == 'blankscreen')),
#     ]

# def to_stims(events, as_dicts=True, blacklist=None):
#     if blacklist is None:
#         blacklist = blacklisttests
#     if not isinstance(blacklist, (tuple, list)):
#         blacklist = (blacklist, )
#     stims = []
#     onscreen = []
#     for e in sorted(events, key=lambda e: e.time):
#         if e.value is None:
#             logging.warning("Encountered event with value == None")
#             if onscreen != {}:
#                 logging.error("Event.value == None with items on screen")
#             continue
#         current = []
#         if hasattr(e.value, '__getitem__'):
#             stimulus = None
#             pixelclock = None
#             for stim in e.value:
#                 if not isinstance(stim, dict) or \
#                         any([t(stim) for t in blacklist]):
#                     continue
#                 if ('name' in stim.keys()) and (stim['name'] == 'pixel clock'):
#                     pixelclock = stim
#                 else:
#                     if stimulus is not None:
#                         logging.warning(
#                             "Two stimuli onscreen: %s, %s"
#                             % (stimulus, stim))
#                     stimulus = stim
#             if stimulus is not None:
#                 current.append(pymworks.events.display.Stimulus(e.time, stimulus, pixelclock))
#         newstims, onscreen = pymworks.events.display.find_stims(onscreen, current, e.time)
#         stims += newstims
#     if as_dicts:
#         return [s.to_dict() for s in stims]
#     return stims

In [8]:
def to_trials(stim_display_events, outcome_events, outcome_key='outcome',
              remove_unknown=True,
              duration_multiplier=2, stim_blacklists=None):
    """
    If remove_unknown, any trials where a corresponding outcome_event cannot
    be found will be removed.
    If duration_multiplier is not None, to_trials will check to see if the
    outcome event occured within duration_multiplier * duration microseconds
    of the trial start. If the outcome event occured later, the trial outcome
    will be marked as unknown.
    """
    if (len(outcome_events) == 0) or (len(stim_display_events) == 0):
        return []
    assert hasattr(outcome_events[0], 'name')

    trials = pymworks.events.display.to_stims(stim_display_events, as_dicts=True,
                      blacklist=stim_blacklists)

    if (len(trials) == 0):
        return []

    outcomes = pymworks.events.utils.sync(outcome_events, trials,
                          direction=1, mkey=lambda x: x['time'])

    assert len(trials) == len(outcomes)
    unknowns = []
    if duration_multiplier is None:
        dtest = lambda t, o: True
    else:
        dtest = lambda t, o: \
            o.time < (t['time'] + t['duration'] * duration_multiplier)
    for i in xrange(len(trials)):
        if (outcomes[i] is not None) and dtest(trials[i], outcomes[i]):
            trials[i]['%s' % outcome_key] = outcomes[i].name
            trials[i]['%s_time' % outcome_key] = outcomes[i].time
        else:
            if remove_unknown:
                unknowns.append(i)
            else:
                trials[i]['%s' % outcome_key] = 'unknown'
                trials[i]['%s_time' % outcome_key] = 'unknown'

    # remove trials with 'unknown' outcome, in reverse
    for u in unknowns[::-1]:
        del trials[u]

    return trials

In [56]:
def parse_trials(dfn, response_types=['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore'], \
                 outcome_types = ['success', 'ignore', 'failure'],\
                 ignore_flags=[], remove_orphans=True):

    print "***** Parsing trials *****"
    df = pymworks.open(dfn)
    
    if ignore_flags is None:
        codec = df.get_codec()
        ignore_flags = []
        all_flags = [f for f in codec.values() if 'Flag' in f or 'flag' in f]
        for fl in all_flags:
            evs = df.get_events(fl)
            vals = list(set([v.value for v in evs]))
            if len(vals) > 1 or len(evs) > 5:
                ignore_flags.append(fl)
        
    # Get run bounds:
    bounds = get_run_time(df)
    if bounds is None:
        return None, None, df

    trials = []; flag_list = []; flags = {};
    for bound in bounds:
        
        if (bound[1]-bound[0])/1E6 < 2.0:
            continue

        # Get display events:
        tmp_devs = df.get_events('#stimDisplayUpdate')                     
        tmp_devs = [i for i in tmp_devs if bound[0] <= i['time']<= bound[1]] 

        # Get behavior flags:
        codec = df.get_codec()
        all_flags = [f for f in codec.values() if 'Flag' in f or 'flag' in f]
        flag_names = [f for f in all_flags if f not in ignore_flags]
        tmp_flags = dict((flag, None) for flag in flag_names)
        for flag in flag_names:
            if flag == 'FlagNoFeedbackInCurrentTrial': continue
            found_values = [e.value for e in df.get_events(flag) if bound[0] <= e.time <=bound[1]]
            if (len(found_values) > 1) or (len(list(set(found_values)))) > 1:
                print("More than 1 value found for flag: %s" % flag)
                tmp_flags[flag] = int(found_values[-1])
            elif (len(found_values) == 1) or (len(list(set(found_values)))) == 1:
                tmp_flags[flag] = int(found_values[0])
            else:
                tmp_flags.pop(flag)
        
        # Add current flag values to flags list:
        flag_list.append(tmp_flags)
        
        # Add boundary time to flag info:
        tmp_flags.update({'run_bounds': bound})

        
        
        # Check for valid response types and get all response events:
        response_types = [r for r in response_types if r in codec.values()]
        response_evs = [e for e in df.get_events(response_types) if (bound[0] < e['time'] < bound[1]) \
                        and e.value!=0]
        #responses = pymworks.events.utils.sync(response_evs, trials, direction=1, mkey=lambda x: x['time'])

        # Convert to trials: match stimulus events and response events:
        outcome_key = 'response'
        responses = to_trials(tmp_devs, response_evs, outcome_key=outcome_key,
                                                   duration_multiplier=2.0,
                                                   stim_blacklists=None,
                                                   remove_unknown=False)
        #print response_types


        # Get OUTCOMES:
        #print outcome_types
    #     outcome_evs = [e for e in df.get_events(outcome_types) if boundary[0] <= e['time']<= boundary[1] and e.value!=0] 
    #     outcomes = pymworks.events.display.to_trials(tmp_devs, outcome_evs,
    #                                                duration_multiplier=1.0,
    #                                                stim_blacklists=None,
    #                                                remove_unknown=False)

        # **sync outcome events to response events as master:
        outcome_evs = [e for e in df.get_events(outcome_types) if (bound[0] < e['time'] < bound[1]) and e.value!=0]
        outcomes = pymworks.events.utils.sync(outcome_evs, responses, direction=1, mkey=lambda x: x['response_time'])

        print "N total response events: ", len(responses)
        print "N total outcome events: ", len(outcomes)

        assert len(responses) == len(outcomes), "**ERROR:  N responses != N outcomes"
        tmp_trials = copy.copy(responses)
        for trial_ix, (response, outcome) in enumerate(zip(responses, outcomes)):
            if outcome is not None:
                tmp_trials[trial_ix].update({'outcome': outcome.name, 'outcome_time': outcome.time}) #['outcome']})
            else:
                tmp_trials[trial_ix].update({'outcome': 'unknown'})

        # Get rid of display events without known outcome within 'duration_multiplier' time
        if remove_orphans:                                                  
            orphans = [(i,x) for i,x in enumerate(tmp_trials) if\
                        x['outcome']=='unknown' or x['%s' % outcome_key]=='unknown']
            tmp_trials = [t for t in tmp_trials if not t['outcome']=='unknown']
            tmp_trials = [t for t in tmp_trials if not t['%s' % outcome_key]=='unknown']

            print "Found and removed %i orphan stimulus events in file %s" % (len(orphans), df.filename)
            print "N valid trials: %i" % len(tmp_trials)
        
        # Add current trials in chunk to trials list:
        trials.extend(tmp_trials)

    # Reformat "name" param for readability:
    if len(trials) == 0:
        return trials, flags, df
    
    for t in trials:
        assert t['response_time'] < t['outcome_time'], "**ERROR: Mismatch in response/outcome alignment"
        stimname = t['name']
        t['name'] = stimname.split('.png')[0]
    
    # Combine all flag states:
    for fi, flag_dict in enumerate(flag_list):
        if fi == 0:
            flags = copy.copy(flag_dict)
        else:
            for flag_name, flag_value in flag_dict.items():
                existing_value = flags[flag_name] #.value()
                if flag_value == existing_value:
                    continue
                if not isinstance(flags[flag_name], list):
                    flags[flag_name] = list(flags[flag_name])
                flags[flag_name].append(flag_value)
                
                
    return trials, flags, df


#### Parse manually using 'next' isntead of pymworks utils:

In [74]:
boundary = get_run_time(df)

devs = [e for e in df.get_events('#stimDisplayUpdate') if boundary[0] < e['time'] < boundary[1] and e.value!=0]

toofast_time = df.get_events('TooFast_time')[-1].value
#print toofast_time


blacklisttests = [
    #lambda s: (('name' in s.keys()) and (s['name'] == 'pixel clock')),
    lambda s: (('type' in s.keys()) and (s['type'] == 'blankscreen')),
    ]

stims = pymworks.events.display.to_stims(devs, as_dicts=True,
                  blacklist=blacklisttests)

# if "aborted" not in response types, check that all durations are > toofast time:
stims = [t for t in stims if t['duration'] > toofast_time] #and t['duration'] >= stim_time]
#print len(stims)

print stims[0]

response_types = ['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore']
response_evs = [e for e in df.get_events(response_types) if boundary[0] < e['time'] < boundary[1] and e.value!=0]
response_evs = sorted(response_evs, key=lambda x: x.time)

outcome_evs = [e for e in df.get_events(outcome_types) if boundary[0] < e['time'] < boundary[1] and e.value!=0]
outcome_evs = sorted(outcome_evs, key=lambda x: x.time)

orphan_stim = []
response_ix = 0
outcome_ix = 0
for trial_ix, stim in enumerate(sorted(stims, key=lambda x: x['time'])):
    try:
        if trial_ix == len(stims)-1:
            curr_response = next(ev for ev in response_evs[response_ix:] if ev.time > stim['time'])
            curr_outcome = next(ev for ev in outcome_evs[outcome_ix:] if ev.time > curr_response.time)
        else:
            curr_response = next(ev for ev in response_evs[response_ix:] if ev.time > stim['time'] \
                                 and ev.time < stims[trial_ix+1]['time'])
            curr_outcome = next(ev for ev in outcome_evs[outcome_ix:] if ev.time > curr_response.time \
                                 and ev.time < stims[trial_ix+1]['time'])
            
        stim.update({'response': curr_response.name, 'response_time': curr_response.time,
                     'outcome': curr_outcome.name})
        
        response_ix = response_evs.index(curr_response) + 1
        outcome_ix = outcome_evs.index(curr_outcome) + 1

    except StopIteration:
        orphan_stim.append(trial_ix)
        #print trial


valid_trials = [i for i in np.arange(0, len(stims)) if i not in orphan_stim]
trials = [stim for stim_ix, stim in enumerate(stims) if stim_ix in valid_trials]
print "N valid trials: %i" % len(trials)


{'pos_x': 0.0, 'pos_y': 0.0, 'name': 'Blob_N2_CamRot_y0.png 1', 'filename': '/var/folders/bq/d_40rj9j1slfyy_dz1h7_7p80000gn/T/MWorks/Experiment Cache/_Users_labuser_Downloads_RatBehaviorGeneral_Behavior_3Dtransforms_2way_blobs/tmp/Blobs_TrainingRatsD1D2/Blob_N1_CamRot_y0.png', 'file_hash': '3c0f591fcbbe5741dbb1b2982cee3e696603f72d', 'duration': 799743, 'size_x': 69.87999725341797, 'size_y': 69.87999725341797, 'time': 187516752582, 'action': 'draw', 'rotation': 0.0, 'type': 'image'}
N valid trials: 805


In [38]:
[t for t, trial in enumerate(trials) if trial['outcome'] is None or trial['response'] is None]

[]

In [18]:
trials, flags = parse_trials(dfn)

['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore']
['success', 'ignore', 'failure']
N total response events:  1358
N total outcome events:  1358
Found and removed 216 orphan stimulus events in file /n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG2_161130.mwk
N valid trials: 1142


#### Check that stim - response - outcome are correct:

In [75]:
trials[0]

{'action': 'draw',
 'duration': 799743,
 'file_hash': '3c0f591fcbbe5741dbb1b2982cee3e696603f72d',
 'filename': '/var/folders/bq/d_40rj9j1slfyy_dz1h7_7p80000gn/T/MWorks/Experiment Cache/_Users_labuser_Downloads_RatBehaviorGeneral_Behavior_3Dtransforms_2way_blobs/tmp/Blobs_TrainingRatsD1D2/Blob_N1_CamRot_y0.png',
 'name': 'Blob_N2_CamRot_y0.png 1',
 'outcome': 'failure',
 'pos_x': 0.0,
 'pos_y': 0.0,
 'response': 'Announce_AcquirePort3',
 'response_time': 187517531099,
 'rotation': 0.0,
 'size_x': 69.87999725341797,
 'size_y': 69.87999725341797,
 'time': 187516752582,
 'type': 'image'}

In [113]:
incorrect_successes = [t for t in trials if 'Blob_N1_CamRot_y' in t['name'] \
                         and t['outcome']=='Announce_AcquirePort3' \
                         and t['result'] != 'success']
print "Blob1: %i incorrect successes" % len(incorrect_successes)

incorrect_successes = [t for t in trials if 'Blob_N2_CamRot_y' in t['name'] \
                         and t['outcome']=='Announce_AcquirePort1' \
                         and t['result'] != 'success']
print "Blob2: %i incorrect successes" % len(incorrect_successes)


ignore_trials = [t for t in trials if t['outcome'] == 'ignore' or t['response'] == 'ignore']
print "N ignores: %i" % len(ignore_trials)

Blob1: 0 incorrect successes
Blob2: 0 incorrect successes
N ignores: 3


In [79]:
df.get_events('ignore')

[Event[code=192, name=ignore, time=187172008460, value=1],
 Event[code=192, name=ignore, time=187172189127, value=1],
 Event[code=192, name=ignore, time=187436297574, value=1],
 Event[code=192, name=ignore, time=188145252478, value=1],
 Event[code=192, name=ignore, time=188174609211, value=1],
 Event[code=192, name=ignore, time=193875666213, value=1]]

#### Test trial and event parsing

In [ ]:
df.get_events('TooFast_time')

In [ ]:
df.get_events('StimulusPresentation_time')

In [ ]:
codec = df.get_codec()
ignore_flags = []
all_flags = [f for f in codec.values() if 'Flag' in f or 'flag' in f]
for fl in all_flags:
    evs = df.get_events(fl)
    vals = list(set([v.value for v in evs]))
    if len(vals) > 1 or len(evs) > 5:
        ignore_flags.append(fl)
ignore_flags

# If there is no "abort" use "Announce_TrialEnd" -- these should be "aborted" trials:
#resp_types=['success', 'failure', 'ignore'] #, 'Announce_TrialEnd']
resp_types=['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore'] #, 'Announce_TrialEnd']

trials, flags = parse_trials(df, resp_types=resp_types, ignore_flags=ignore_flags)


In [ ]:
trials[0]

In [ ]:
successes = [d for d in df.get_events('success') if boundary[0] < d.time < boundary[1]]
failures = [d for d in df.get_events('failure') if boundary[0] < d.time < boundary[1]]
ignores = [d for d in df.get_events('ignore') if boundary[0] < d.time < boundary[1]]
ntotal = sum([len(successes), len(failures), len(ignores)])

print("Success: %i, Failure: %i, Ignore: %i" % (len(successes), len(failures), len(ignores)))
print("TOTAL: %i" % ntotal)


In [ ]:

successes = [t for t in trials if t['outcome'] == 'success']
failures = [t for t in trials if t['outcome'] == 'failure']
ignores = [t for t in trials if t['outcome'] == 'ignore']

ntotal = len(trials)

print("Success: %i, Failure: %i, Ignore: %i" % (len(successes), len(failures), len(ignores)))
print("TOTAL: %i" % ntotal)


In [ ]:
choice_types = ['success', 'failure', 'ignore']
aborted = [t for t in trials if t['outcome'] not in choice_types]
print("Aborted: %i" % len(aborted))

In [ ]:
ignore_evs = [d for d in df.get_events('ignore') if boundary[0] <= d.time <= boundary[1]]

In [ ]:
boundary

In [ ]:
[(d.time-1903319378052 ) / 1E6 for d in ignore_evs]

##### Identify NoFeedback Trials:

In [ ]:
fb_evs = [e for e in df.get_events('FlagNoFeedbackInCurrentTrial') if boundary[0] <= e.time <=boundary[1]]

In [ ]:
df.get_events('FlagShowStimAfterResponse')

In [ ]:
for tix, t in enumerate(trials):
    dur_s = t['duration']/1E6
    if dur_s < 0.38:
        print tix, t['outcome']

In [ ]:
for tix, trial in enumerate(trials[0:20]):
    ttime = (trial['time'], trial['time'] + trial['duration'])
    nofeedback = [e.value for e in df.get_events('FlagNoFeedbackInCurrentTrial') if \
                  ttime[0]-1000000 <= e.time <=ttime[1] ] #and e.value==1]
    print tix, nofeedback #len(nofeedback)

In [ ]:
fb_evs[0:100]

### Group trials by stimulus type:

In [10]:
def get_session_meta(dfns): #, resp_types=[], remove_orphans=True, n_training_phases=5):
    
    sessions_info = {}
    for dfn in sorted(dfns, key=natural_keys):
        aid, datestr = parse_datafile_name(dfn)

        if datestr in sessions_info.keys():
            if not isinstance(sessions_info[datestr]['datasource'], list):
                sessions_info[datestr]['datasource'] = [sessions_info[datestr]['datasource']]
            sessions_info[datestr]['datasource'].append(dfn)
        else:
            sessions_info[datestr] = dict()
            sessions_info[datestr]['datasource'] = [dfn] 
            
        sessions_info[datestr]['animal'] = aid
        sessions_info[datestr]['session'] = datestr
    
    return sessions_info

In [11]:
class Session():
    def __init__(self, session_meta):  
        #animalid, datestr = parse_datafile_name(dfn)
        self.name = session_meta['animal'] #animalid
        self.session = session_meta['session'] #datestr
        self.source = session_meta['datasource'] #dfn
        self.experiment = None
        self.protocol = None
        self.server = None # session_meta['server']
        self.trials = None
        self.flags = None
        self.stimuli = None
        self.stats = None
        self.summary = None
        
    def parse_trials(self, 
                     ignore_flags=None,
                     response_types=['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore'],
                     outcome_types=['success', 'failure', 'ignore']):
                    # If there is no "abort" use "Announce_TrialEnd" ? -- these should be "aborted" trials?
        
        trials = []
        flags = {}
        if isinstance(self.source, list) and len(self.source) > 1:
            tmp_flags = []
            for dfn in self.source:
                curr_trials, curr_flags, df = parse_trials(dfn, response_types=response_types, 
                                                       outcome_types=outcome_types,
                                                       ignore_flags=ignore_flags)
                if curr_trials is not None:
                    trials.extend(curr_trials)
                    tmp_flags.append(curr_flags)

            # Combine flag values across data files:
            if len(tmp_flags) > 0:
                flags = dict((fkey, []) for fkey in tmp_flags[0].keys())
                for tmp_flag in tmp_flags:
                    for flag_key, flag_value in tmp_flag.iteritems():
                        if flag_key not in flags.keys():
                            flags[flag_key] = []
                        if flag_value not in flags[flag_key]:
                            flags[flag_key].append(flag_value)
        else:
            # Open data file:
            dfn = self.source[0]
            trials, flags, df = parse_trials(dfn, response_types=response_types, 
                                         outcome_types=outcome_types,
                                         ignore_flags=ignore_flags)

        self.trials = trials
        self.flags = flags

        # Get current session server info while df open:
        server_address = df.get_events('#serialBridgeAddress')[-1].value
        server_name = df.get_events('#serverName')[-1].value
        self.server =  {'address': server_address, 'name': server_name}
        
        # Save experiment and protocol info:
        # [(payload_type, event_type)]:  [(4013, 1002), (2001, 1001), (4007, 1002)] # 1002:  Datafile creation
        experiment_load = 4013 #:  Looks like which experiment file(s) loaded (.mwk)
        protocol_load = 2001 #:  Which protocols found and which loaded
        sys_evs = df.get_events('#systemEvent')
        
        # Get experiment loaded:
        exp_evs = [v for v in sys_evs if v.value['payload_type']==experiment_load]
        exp_path = list(set([v.value['payload']['experiment path'] for v in exp_evs]))
        #assert len(exp_path) == 1, "*ERROR* More than 1 experiment loaded..."
        #exp_path = exp_path[0].split('/Experiment Cache/')[1]
        self.experiment_path = exp_path

        # Get protocol loaded:
        prot_evs = [v for v in sys_evs if v.value['payload_type']==protocol_load]
        protocol = list(set([v.value['payload']['current protocol'] for v in prot_evs]))
        #assert len(protocol) == 1, "*ERROR* More than 1 protocol loaded..."
        #protocol = protocol[0]
        self.protocol = protocol
        
        
    def get_counts_by_stimulus(self):
        print("... Getting stimulus counts ...")
        stats = None
        if self.trials is not None:
            by_stim = datautils.grouping.group(self.trials, 'name')

            ordered_stim = sorted(by_stim.keys(), key=lambda x: x.split('_')[-1][1:])
            #print ordered_stim

            stats = dict((stim, {}) for stim in by_stim.keys())
            for stim in by_stim.keys():
                stats[stim]['ntrials'] = len(by_stim[stim])
                stats[stim]['nsuccess'] = sum([1 if trial['outcome']=='success' else 0 for trial in by_stim[stim]])
                stats[stim]['nfailure'] = sum([1 if trial['outcome']=='failure' else 0 for trial in by_stim[stim]])
                stats[stim]['nignore'] = sum([1 if trial['outcome']=='ignore' else 0 for trial in by_stim[stim]])
                stats[stim]['nchoose1'] = sum([1 if trial['response']=='Announce_AcquirePort1' else 0 for trial in by_stim[stim]])
                stats[stim]['nchoose3'] = sum([1 if trial['outcome']=='Announce_AcquirePort3' else 0 for trial in by_stim[stim]])

            # Save stimulus names for easy access:
            stimulus_names = list(set([trial['name'] for trial in self.trials]))
            self.stimuli = stimulus_names
        
        self.stats = stats

        
    def get_summary(self):
        print("... Getting session summary ...")
        # Get stats by stim, if not run:
        if self.stats is None:
            self.get_counts_by_stimulus()
        
        if self.stats is not None:
            summary_keys = ['ntrials', 'nsuccess', 'nfailure', 'nignore']
            summary = dict((k, 0) for k in summary_keys)
            if len(self.stats.keys()) > 0:
                for stat in summary.keys():
                    summary[stat] = [val[stat] for stim, val in self.stats.items()]
            self.summary = summary
        
    def plot_stats_by_transform(self, output_figdir='/tmp'):
        print("... Plotting stats by transform ...")
        stats = self.stats
        
        # Check if there are morphs also:
        morph_list = [s for s in stats.keys() if 'morph' in s]
        if len(morph_list) > 0:
            stimulus_list = [s for s in stats.keys() if s not in morph_list]
        else:
            stimulus_list = stats.keys()
            
        # Check stimulus name:
        blob_names = [b for b in self.stimuli if 'Blob_' in b]
        if 'N' in blob_names[0].split('_')[1]: # this is Blob_Nx_CamRot naming scheme:
            blob1_name = 'Blob_N1'
            blob2_name = 'Blob_N2'
        else:
            blob1_name = 'Blob_1'
            blob2_name = 'Blob_2'
            
        if stats is not None:
            values = [('%s_%s' % ('_'.join(stim.split('_')[0:2]), stim.split('_')[3]), \
                       stats[stim]['nsuccess']/float(stats[stim]['ntrials'])) for stim in stimulus_list]
            print values

            pl.figure()
            if 'CamRot' in stimulus_list[0]:
                blob1 = [(int(v[0].split('_')[-1][1:]), v[1]) for v in values if blob1_name in v[0]]
                blob2 = [(int(v[0].split('_')[-1][1:]), v[1]) for v in values if blob2_name in v[0]]
            else:
                blob1 = [(int(v[0].split('_')[-1]), v[1]) for v in values if blob1_name in v[0]]
                blob2 = [(int(v[0].split('_')[-1]), v[1]) for v in values if blob2_name in v[0]]
                
            pl.plot([b[0] for b in sorted(blob1, key=lambda x: x[0])], \
                    [b[1] for b in sorted(blob1, key=lambda x: x[0])], 'ro', label=blob1_name)
            pl.plot([b[0] for b in sorted(blob2, key=lambda x: x[0])], \
                    [b[1] for b in sorted(blob2, key=lambda x: x[0])], 'bo', label=blob2_name)
            pl.legend()
            pl.ylim(0, 1)
            pl.title(datestr)
            pl.savefig(os.path.join(output_figdir, '%s_%s_bystim.png' % (self.name, self.session)))
            pl.close()

    def plot_stats_by_morph(self, output_figdir='/tmp'):
        print("... Plotting stats by transform ...")

        stats = self.stats
        stimulus_list = [s for s in stats.keys() if 'morph' in s]
            
        if stats is not None:
            values = [('morph_%s' % (stim.split('morph')[1]), \
                       stats[stim]['nchoose1']/float(stats[stim]['ntrials'])) for stim in stimulus_list]

            pl.figure()
            pchoose1 = [(int(v[0].split('_')[-1]), v[1]) for v in values]
            pl.plot([b[0] for b in sorted(pchoose1, key=lambda x: x[0])], \
                    [b[1] for b in sorted(pchoose1, key=lambda x: x[0])], 'bo')
            pl.ylim(0, 1)
            pl.ylabel("perc. choose port 1)")
            pl.title(datestr)
            pl.savefig(os.path.join(output_figdir, '%s_%s_morphs.png' % (self.name, self.session)))
            pl.close()



In [12]:
class Animal():
    def __init__(self, animalid='RAT', experiment='EXPERIMENT', output_datadir='/tmp'):
        self.animalid = animalid
        self.experient = experiment
        self.outdir = output_datadir
        self.sessions = {}
        
    def get_sessions(self, dfns):
        session_info = get_session_meta(dfns)
        return session_info


In [13]:
def process_session(session_meta, 
                    output_figdir='/tmp',
                    response_types=['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore'],
                    outcome_types=['success', 'ignore', 'failure'],
                    ignore_flags=None):
    
    S = Session(session_meta)
    print session_meta
    S.parse_trials(response_types=['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore'], \
                 outcome_types = ['success', 'ignore', 'failure'])

    S.get_summary() #S.get_counts_by_stimulus()

    if S.summary is None or S.summary['ntrials'] == 0:
        return None
    
    S.plot_stats_by_transform(output_figdir=output_figdir)

    if any('morph' in i for i in S.stimuli):
        S.plot_stats_by_morph(output_figdir=output_figdir)
        
    # Save tmp file:
    tmp_file_dir = os.path.join(output_figdir, 'tmp_files')
    if not os.path.exists(tmp_file_dir): os.makedirs(tmp_file_dir)
    with open(os.path.join(tmp_file_dir, 'proc_%s_%s.pkl' % (S.name, S.session)), 'wb') as f:
        pkl.dump(S, f, protocol=pkl.HIGHEST_PROTOCOL)
        
    return S

In [55]:
def process_sessions_mp(new_sessions, session_info,
                         output_figdir='/tmp',
                         nprocesses=1,
                         ignore_flags=None,
                         response_types=['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore'],
                         outcome_types = ['success', 'ignore', 'failure']):
    
    print "SAVING FIGURES TO:", output_figdir
    
    def parser(curr_sessions, session_info, ignore_flags, response_types, outcome_types, output_figdir, out_q):
        parsed_sessions = {}
        for datestr in curr_sessions:
            session_meta = session_info[datestr]
            S = process_session(session_meta, 
                                output_figdir=output_figdir,
                                ignore_flags=ignore_flags,
                                response_types=response_types, 
                                outcome_types=outcome_types)
            parsed_sessions[datestr] = S
        out_q.put(parsed_sessions)
    
    # Get a chunksize of sessions to process and queue for outputs:
    out_q = mp.Queue()
    chunksize = int(math.ceil(len(new_sessions) / float(nprocesses)))
    procs = []
    for i in range(nprocesses):
        p = mp.Process(target=parser,
                      args=(new_sessions[chunksize * i:chunksize * (i + 1)],
                           session_info, 
                           ignore_flags,
                           response_types,
                           outcome_types,
                           output_figdir,
                           out_q))
        procs.append(p)
        p.start()
        
    # Collect all results into single dict:
    processed_dict = {}
    for i in range(nprocesses):
        processed_dict.update(out_q.get())
    
    # Wait for all worker processes to finish:
#     for p in procs:
#         print "Finished:", p
#         p.join()
    TIMEOUT = 60
    start = time.time()
    while time.time() - start <= TIMEOUT:
        if any([p.is_alive() for p in procs]):
            time.sleep(.1)
        else:
            break # all processes complete, break
    else:
        # kill processes if time out
        print("timed out... killing all processes.")
        for p in procs:
            p.terminate()
            p.join()
            
        
    return processed_dict

# Parse .mwk data files

#### 1. Set sources and output dirs:

In [38]:
# Set path params:
root = '/n/coxfs01/behavior-data'
experiment = 'threeport_morphs'
cohort = 'AG'

# Set experiment parsing vars and params:
response_types = ['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore']
outcome_types = outcome_types = ['success', 'ignore', 'failure']
ignore_flags = None


In [39]:
# Create output dirs:
output_dir = os.path.join(root, experiment, cohort, 'processed')
output_figdir = os.path.join(output_dir, 'figures')
output_datadir = os.path.join(output_dir, 'data')
if not os.path.exists(output_figdir): os.makedirs(output_figdir)
if not os.path.exists(output_datadir): os.makedirs(output_datadir)


print("Saving parsed datafiles to: %s" % output_datadir)
print("Saving figures to: %s" % output_figdir)

Saving parsed datafiles to: /n/coxfs01/behavior-data/threeport_morphs/AG/processed/data
Saving figures to: /n/coxfs01/behavior-data/threeport_morphs/AG/processed/figures


#### 2. Get list of all datafiles for each animal in cohort
Datafile format:  ANIMALID_YYMMDD.mwk (convert datestr to YYYYMMDD)


In [40]:
# Get list of a.. raw data files:
raw_fns = glob.glob(os.path.join(root, experiment, cohort, 'raw', '*.mwk'))

sessions = {}
for fn in raw_fns:
    animalid, datestr = parse_datafile_name(fn)
    if animalid not in sessions.keys():
        sessions[animalid] = []
    sessions[animalid].append(int(datestr))
    
    
animalids = sorted(sessions.keys(), key=natural_keys)
nsessions = [len(sessions[animal]) for animal in sorted(animalids, key=natural_keys)]
all_sessions = sorted([int(item) for sublist in sessions.values() for item in sublist])

print('----------------------------------------------')
print("Found %i animals in cohort %s." % (len(animalids), cohort))
print("-- first session: %i, last session: %i --" % (all_sessions[0], all_sessions[-1]))
print("N sessions per animal:")
for animalid, session_count in zip(animalids, nsessions):
    print("%s: %i sessions" % (animalid, session_count))
print('----------------------------------------------')


----------------------------------------------
Found 12 animals in cohort AG.
-- first session: 20150202, last session: 20170202 --
N sessions per animal:
AG1: 205 sessions
AG2: 196 sessions
AG3: 29 sessions
AG4: 156 sessions
AG5: 172 sessions
AG6: 173 sessions
AG7: 189 sessions
AG8: 190 sessions
AG9: 132 sessions
AG10: 155 sessions
AG11: 155 sessions
AG12: 42 sessions
----------------------------------------------


#### 3.  Process new datafiles for each animal in cohort

In [82]:
# AG2:  
# up thru 20150313 - Blob_1_RotDept_0
# starting 20150314 -- Blob_1_CamRot_y0
# morphs start 20160623

animalid = 'AG3'

session_list = sessions[animalid]




In [83]:
# Get current animal session info:
animal = Animal(animalid=animalid, experiment=experiment, output_datadir=output_datadir)
curr_dfns = [dfn for dfn in raw_fns if animalid == os.path.splitext(os.path.split(dfn)[-1])[0].split('_')[0] or animalid == os.path.splitext(os.path.split(dfn)[-1])[0].split('_')[1]]
session_info = animal.get_sessions(curr_dfns)

# Create animal datafile:
animal_datafile = os.path.join(output_datadir, '%s.pkl' % animalid)


In [84]:
session_info

{'20150203': {'animal': 'AG3',
  'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_150203.mwk'],
  'session': '20150203'},
 '20150204': {'animal': 'AG3',
  'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_150204.mwk'],
  'session': '20150204'},
 '20160127': {'animal': 'AG3',
  'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160127.mwk'],
  'session': '20160127'},
 '20160713': {'animal': 'AG3',
  'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160713.mwk'],
  'session': '20160713'},
 '20160714': {'animal': 'AG3',
  'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160714.mwk'],
  'session': '20160714'},
 '20160715': {'animal': 'AG3',
  'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160715.mwk'],
  'session': '20160715'},
 '20160716': {'animal': 'AG3',
  'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160716.mwk'],
  'session': '20160716'},
 '2016

In [85]:
[s for s, sd in session_info.items() if sd['animal'] != animalid]

[]

In [86]:
# Check if processed file exists -- load or create new.
create_new = False
if os.path.exists(animal_datafile):
    try:
        with open(animal_datafile, 'rb') as f:
            animal = pkl.load(f)   
    except EOFError:
        create_new = True
        
if create_new:
    animal = Animal(animalid=animalid, experiment=experiment, output_datadir=output_datadir)


# Process new datafiles / sessions:
old_sessions = [sesh for sesh, sobject in animal.sessions.items() if sobject is not None]
print("[%s]: Found %i processed sessions." % (animalid, len(old_sessions)))
all_sessions = session_info.keys()
new_sessions = [s for s in all_sessions if s not in old_sessions]

print("[%s]: There are %i out of %i found session datafiles to process." % (animal.animalid, len(new_sessions), len(all_sessions)))


[AG3]: Found 0 processed sessions.
[AG3]: There are 29 out of 29 found session datafiles to process.


In [87]:
# Process all new sessions:
processed_sessions = process_sessions_mp(new_sessions, session_info,
                                         output_figdir=output_figdir,
                                         nprocesses=4,
                                         ignore_flags=ignore_flags,
                                         response_types=response_types,
                                         outcome_types=outcome_types)

SAVING FIGURES TO: /n/coxfs01/behavior-data/threeport_morphs/AG/processed/figures
{'session': '20160127', 'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160127.mwk'], 'animal': 'AG3'}
***** Parsing trials *****
{'session': '20160725', 'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160725.mwk'], 'animal': 'AG3'}
***** Parsing trials *****
{'session': '20160722', 'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160722.mwk'], 'animal': 'AG3'}
***** Parsing trials *****
{'session': '20160804', 'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160804.mwk'], 'animal': 'AG3'}
***** Parsing trials *****


N total response events:  969
N total outcome events:  969
Found and removed 79 orphan stimulus events in file /n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160127.mwk
N valid trials: 890
... Getting session summary ...
... Getting stimulus counts ...
... Plotting stats by transform ...
[('Blob_N2_y45', 1.0), ('Blob_N1_y60', 0.7777777777777778), ('Blob_N1_y-15', 0.85), ('Blob_N2_y-30', 0.8709677419354839), ('Blob_N1_y45', 0.8823529411764706), ('Blob_N1_y-15', 0.6666666666666666), ('Blob_N2_y-15', 0.7058823529411765), ('Blob_N2_y30', 0.72), ('Blob_N2_y0', 0.9545454545454546), ('Blob_N1_y0', 0.75), ('Blob_N2_y-30', 0.6470588235294118), ('Blob_N1_y45', 0.5714285714285714), ('Blob_N2_y30', 0.8333333333333334), ('Blob_N2_y60', 0.6818181818181818), ('Blob_N2_y0', 0.6111111111111112), ('Blob_N2_y-30', 0.875), ('Blob_N2_y-15', 0.8846153846153846), ('Blob_N2_y45', 0.6190476190476191), ('Blob_N1_y0', 0.21739130434782608), ('Blob_N1_y30', 0.8333333333333334), ('Blob_N1_y0', 0.6842105263157

N total response events:  928
N total outcome events:  928
Found and removed 89 orphan stimulus events in file /n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160804.mwk
N valid trials: 839
... Getting session summary ...
... Getting stimulus counts ...
... Plotting stats by transform ...
[('Blob_N2_y60', 0.4), ('Blob_N2_y45', 0.3333333333333333), ('Blob_N1_y45', 0.5333333333333333), ('Blob_N1_y60', 0.4666666666666667), ('Blob_N2_y15', 0.3), ('Blob_N1_y-30', 0.6), ('Blob_N1_y-15', 0.6666666666666666), ('Blob_N1_y0', 0.5786516853932584), ('Blob_N2_y30', 0.5517241379310345), ('Blob_N1_y-45', 0.45161290322580644), ('Blob_N2_y-45', 0.5862068965517241), ('Blob_N2_y0', 0.430939226519337), ('Blob_N1_y30', 0.6), ('Blob_N2_y-60', 0.5), ('Blob_N1_y15', 0.7666666666666667), ('Blob_N2_y-15', 0.43333333333333335), ('Blob_N1_y-60', 0.4666666666666667), ('Blob_N2_y-30', 0.45161290322580644)]


{'session': '20160803', 'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160803.mwk'], 'animal': 'AG3'}
***** Parsing trials *****
N total response events:  637
N total outcome events:  637
Found and removed 327 orphan stimulus events in file /n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_150204.mwk
N valid trials: 310
... Getting session summary ...
... Getting stimulus counts ...
... Plotting stats by transform ...
[('Blob_2_0', 0.49390243902439024), ('Blob_1_0', 0.5205479452054794)]
{'session': '20150203', 'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_150203.mwk'], 'animal': 'AG3'}
***** Parsing trials *****


More than 1 value found for flag: FlagAutomaticBiasSuppression
More than 1 value found for flag: FlagStaircaseHorizSeparation
More than 1 value found for flag: FlagStaircaseProbFreeRew
More than 1 value found for flag: FlagEnrichSomeStaircases
More than 1 value found for flag: FlagForceCueStimSoundOff
More than 1 value found for flag: FlagShowOnlyTrainedAxes
More than 1 value found for flag: FlagDiscardHalfOfTheTrials
More than 1 value found for flag: FlagDiscardTrialWithCurrentDeptRot
More than 1 value found for flag: FlagSampUniformDistr
More than 1 value found for flag: FlagStaircaseSize
More than 1 value found for flag: FlagStaircasePosHR
More than 1 value found for flag: FlagStaircasePosHL
More than 1 value found for flag: FlagStaircasePosVU
More than 1 value found for flag: FlagStaircasePosVD
More than 1 value found for flag: FlagStaircaseRotCW
More than 1 value found for flag: FlagStaircaseRotACW
More than 1 value found for flag: FlagStaircaseDeptRotRight
More than 1 value found

...

N total response events:  1199
N total outcome events:  1199
Found and removed 177 orphan stimulus events in file /n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160726.mwk
N valid trials: 1022
... Getting session summary ...
... Getting stimulus counts ...
... Plotting stats by transform ...
[('Blob_N2_y-15', 0.2857142857142857), ('Blob_N1_y0', 0.6796875), ('Blob_N2_y-30', 0.3181818181818182), ('Blob_N2_y0', 0.3515625), ('Blob_N1_y-60', 0.7142857142857143), ('Blob_N2_y15', 0.47619047619047616), ('Blob_N2_y30', 0.42857142857142855), ('Blob_N1_y30', 0.7142857142857143), ('Blob_N1_y15', 0.7272727272727273), ('Blob_N1_y-30', 0.8571428571428571), ('Blob_N1_y-15', 0.6818181818181818), ('Blob_N2_y-45', 0.47619047619047616), ('Blob_N2_y-60', 0.23809523809523808), ('Blob_N2_y45', 0.5714285714285714), ('Blob_N1_y-45', 0.7142857142857143), ('Blob_N1_y45', 0.7272727272727273), ('Blob_N1_y60', 0.6666666666666666), ('Blob_N2_y60', 0.2857142857142857)]
N total response events:  1074
N total out

N total response events:  1491
N total outcome events:  1491
Found and removed 170 orphan stimulus events in file /n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160813.mwk
N valid trials: 1321
... Getting session summary ...
... Getting stimulus counts ...
... Plotting stats by transform ...
[('Blob_N2_y-15', 0.5), ('Blob_N1_y0', 0.5767634854771784), ('Blob_N2_y-30', 0.525), ('Blob_N1_y-60', 0.525), ('Blob_N2_y15', 0.325), ('Blob_N2_y30', 0.5), ('Blob_N1_y30', 0.5), ('Blob_N1_y15', 0.675), ('Blob_N1_y-30', 0.675), ('Blob_N1_y-15', 0.55), ('Blob_N2_y-45', 0.425), ('Blob_N2_y-60', 0.525), ('Blob_N2_y45', 0.525), ('Blob_N1_y-45', 0.725), ('Blob_N1_y45', 0.75), ('Blob_N1_y60', 0.5), ('Blob_N2_y60', 0.475), ('Blob_N2_y0', 0.5166666666666667)]
... Plotting stats by transform ...
{'session': '20160814', 'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160814.mwk'], 'animal': 'AG3'}
***** Parsing trials *****
N total response events:  1214
N total outcome events:  121

N total response events:  808
N total outcome events:  808
Found and removed 74 orphan stimulus events in file /n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160816.mwk
N valid trials: 734
... Getting session summary ...
... Getting stimulus counts ...
... Plotting stats by transform ...
[('Blob_N2_y-15', 0.5217391304347826), ('Blob_N1_y0', 0.696969696969697), ('Blob_N1_y-45', 0.8636363636363636), ('Blob_N2_y-30', 0.45454545454545453), ('Blob_N2_y0', 0.41911764705882354), ('Blob_N1_y-60', 0.8636363636363636), ('Blob_N2_y15', 0.4090909090909091), ('Blob_N2_y30', 0.4090909090909091), ('Blob_N1_y30', 0.6363636363636364), ('Blob_N1_y15', 0.6363636363636364), ('Blob_N1_y-30', 0.7272727272727273), ('Blob_N1_y-15', 0.8181818181818182), ('Blob_N2_y-45', 0.34782608695652173), ('Blob_N2_y-60', 0.22727272727272727), ('Blob_N2_y45', 0.2727272727272727), ('Blob_N1_y45', 0.6818181818181818), ('Blob_N1_y60', 0.7391304347826086), ('Blob_N2_y60', 0.4090909090909091)]
... Plotting stats by transfo

N total response events:  1545
N total outcome events:  1545
Found and removed 282 orphan stimulus events in file /n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160815.mwk
N valid trials: 1263
... Getting session summary ...
... Getting stimulus counts ...
... Plotting stats by transform ...
[('Blob_N2_y-15', 0.41025641025641024), ('Blob_N1_y0', 0.6826086956521739), ('Blob_N1_y-45', 0.6578947368421053), ('Blob_N2_y-30', 0.47368421052631576), ('Blob_N2_y0', 0.48695652173913045), ('Blob_N1_y-60', 0.7692307692307693), ('Blob_N2_y15', 0.5641025641025641), ('Blob_N2_y30', 0.3684210526315789), ('Blob_N1_y30', 0.631578947368421), ('Blob_N1_y15', 0.5), ('Blob_N1_y-30', 0.5789473684210527), ('Blob_N1_y-15', 0.6578947368421053), ('Blob_N2_y-45', 0.34210526315789475), ('Blob_N2_y-60', 0.34210526315789475), ('Blob_N2_y45', 0.2894736842105263), ('Blob_N1_y45', 0.7105263157894737), ('Blob_N1_y60', 0.6153846153846154), ('Blob_N2_y60', 0.47368421052631576)]
... Plotting stats by transform ...
{'

N total response events:  979
N total outcome events:  979
Found and removed 127 orphan stimulus events in file /n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG3_160817.mwk
N valid trials: 852
... Getting session summary ...
... Getting stimulus counts ...
... Plotting stats by transform ...
[('Blob_N2_y-15', 0.4230769230769231), ('Blob_N1_y0', 0.6089743589743589), ('Blob_N1_y-45', 0.8), ('Blob_N2_y-30', 0.5769230769230769), ('Blob_N2_y0', 0.6209150326797386), ('Blob_N1_y-60', 0.7692307692307693), ('Blob_N2_y15', 0.38461538461538464), ('Blob_N2_y30', 0.48), ('Blob_N1_y30', 0.6153846153846154), ('Blob_N1_y15', 0.56), ('Blob_N1_y-30', 0.6538461538461539), ('Blob_N1_y-15', 0.7692307692307693), ('Blob_N2_y-45', 0.5), ('Blob_N2_y-60', 0.5769230769230769), ('Blob_N2_y45', 0.34615384615384615), ('Blob_N1_y45', 0.7037037037037037), ('Blob_N1_y60', 0.48), ('Blob_N2_y60', 0.5)]
... Plotting stats by transform ...


In [74]:
# Update animal sessions dict:
for datestr, S in processed_sessions.items():
    animal.sessions.update({datestr: S})

# Save to disk:
try:
    with open(animal_datafile, 'wb') as f:
        pkl.dump(animal, f, protocol=pkl.HIGHEST_PROTOCOL)
except PicklingError:
    print("Unable to pkl: New sessions are not the same class as old sessions.")
    print("Reprocessing %i old sessions..." % len(processed_sessions))
    for datestr in old_sessions:
        session_meta = session_info[datestr]
        S = process_session(session_meta)
        animal.sessions[datestr] = S
        
    with open(animal_datafile, 'wb') as f:
        pkl.dump(animal, f, protocol=pkl.HIGHEST_PROTOCOL)

print("[%s] ~~~ processing complete! ~~~" % animal.animalid)



[AG2] ~~~ processing complete! ~~~


In [74]:
# Reprocess buggy datafiles:

# processed_sessions = {}
# for datestr in new_sessions:
#     session_meta = session_info[datestr]
#     S = process_session(session_meta, 
#                         output_figdir=output_figdir,
#                         ignore_flags=ignore_flags,
#                         response_types=response_types, 
#                         outcome_types=outcome_types)
#     processed_sessions[datestr] = S

{'session': '20151119', 'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG2_151119.mwk'], 'animal': 'AG2'}
***** Parsing trials *****
... Getting session summary ...
... Getting stimulus counts ...
{'session': '20151118', 'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG2_151118.mwk'], 'animal': 'AG2'}
***** Parsing trials *****
... Getting session summary ...
... Getting stimulus counts ...
{'session': '20151120', 'datasource': ['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG2_151120.mwk'], 'animal': 'AG2'}
***** Parsing trials *****
N total response events:  0
N total outcome events:  0
Found and removed 0 orphan stimulus events in file /n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG2_151120.mwk
N valid trials: 0
... Getting session summary ...
... Getting stimulus counts ...


#### Save processed sessions to animal object in case MP errors out:

In [75]:
tmp_processed_sessions = glob.glob(os.path.join(output_figdir, 'tmp_files', 'proc_%s*.pkl' % animal.animalid))
print("Found %i processed sessions in tmp dir." % len(tmp_processed_sessions))

for tmpfile in tmp_processed_sessions:
    with open(tmpfile, 'rb') as f:
        tmpS = pkl.load(f)
    datestr = os.path.splitext(os.path.split(tmpfile)[-1])[0].split('_')[2]
    #print datestr
    animal.sessions.update({datestr: tmpS})



# Save to disk:
try:
    with open(animal_datafile, 'wb') as f:
        pkl.dump(animal, f, protocol=pkl.HIGHEST_PROTOCOL)
except PicklingError:
    print("Unable to pkl: New sessions are not the same class as old sessions.")
    print("Reprocessing %i old sessions..." % len(processed_sessions))
    for datestr in old_sessions:
        session_meta = session_info[datestr]
        S = process_session(session_meta)
        animal.sessions[datestr] = S
        
    with open(animal_datafile, 'wb') as f:
        pkl.dump(animal, f, protocol=pkl.HIGHEST_PROTOCOL)

print("[%s] ~~~ processed session update complete! ~~~" % animal.animalid)



Found 192 processed sessions in tmp dir.
[AG2] ~~~ processed session update complete! ~~~


#### Checking funky files:

In [234]:
fn = '/media/julianarhee/BK/mworks_data/AG/AG2_150803.mwk'
df = pymworks.open(fn)



In [235]:
print "response types:", response_types
print "outcome_types:", outcome_types

response types: ['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore']
outcome_types: ['success', 'ignore', 'failure']


In [236]:
trials, flags = parse_trials(df, response_types=response_types, ignore_flags=ignore_flags)


['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore']
['success', 'ignore', 'failure']
N total response events:  0
N total outcome events:  0
Found and removed 0 orphan stimulus events in file /media/julianarhee/BK/mworks_data/AG/AG2_150803.mwk
N valid trials: 0


In [241]:
boundary = get_run_time(df)
(boundary[1] - boundary[0]) /1E6

1.895642

In [243]:
df.get_events('#state_system_mode')

[Event[code=5, name=#state_system_mode, time=422098815072, value=0],
 Event[code=5, name=#state_system_mode, time=422098998966, value=0],
 Event[code=5, name=#state_system_mode, time=422131902902, value=2],
 Event[code=5, name=#state_system_mode, time=422131902909, value=2],
 Event[code=5, name=#state_system_mode, time=422133798544, value=1],
 Event[code=5, name=#state_system_mode, time=422133799168, value=0],
 Event[code=5, name=#state_system_mode, time=422135401853, value=2],
 Event[code=5, name=#state_system_mode, time=422135401859, value=2],
 Event[code=5, name=#state_system_mode, time=428988902033, value=1],
 Event[code=5, name=#state_system_mode, time=428988902618, value=0]]

#### Testing / debugging Session objects:

In [190]:
curr_session = 20150314 #20160623 #session_list[pre_morphs] # 20150211

datestr = session_list[session_list.index(curr_session)]
dfn = [f for f in raw_fns if animalid in f and str(datestr)[2:] in f][0]
print dfn

/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG2_150314.mwk


In [153]:
# df = pymworks.open(dfn)
# trials, flags = parse_trials(df)

['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore']
['success', 'ignore', 'failure']
N total response events:  1051
N total outcome events:  1051
Found and removed 110 orphan stimulus events in file /n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG2_160623.mwk
N valid trials: 941


In [191]:
S = Session(dfn)
S.parse_trials(response_types=['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore'], \
                 outcome_types = ['success', 'ignore', 'failure'])

['Announce_AcquirePort1', 'Announce_AcquirePort3', 'ignore']
['success', 'ignore', 'failure']
N total response events:  335
N total outcome events:  335
Found and removed 57 orphan stimulus events in file /n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG2_150314.mwk
N valid trials: 278


In [192]:
S.get_counts_by_stimulus()

In [193]:
# Check if there are morphs also:
morph_list = [s for s in S.stats.keys() if 'morph' in s]
if len(morph_list) > 0:
    stimulus_list = [s for s in S.stats.keys() if s not in morph_list]
else:
    stimulus_list = S.stats.keys()
print stimulus_list

['Blob_N2_CamRot_y0', 'Blob_N1_CamRot_y0']


In [194]:


values = [('%s_%s' % (stim.split('_')[1], stim.split('_')[3]), \
                       S.stats[stim]['nsuccess']/float(S.stats[stim]['ntrials'])) for stim in stimulus_list]

In [195]:
values

[('N2_y0', 0.9496402877697842), ('N1_y0', 0.9136690647482014)]

In [142]:
#stimulus_list = S.stimuli
stimulus_list = [s for s in S.stats.keys() if 'morph' in s]

print stimulus_list

['morph0', 'morph1', 'morph3', 'morph5', 'morph6', 'morph7', 'morph8', 'morph9', 'morph12', 'morph13', 'morph10', 'morph11', 'morph16', 'morph14', 'morph15', 'morph2', 'morph4']


In [147]:
values = [('morph_%s' % (stim.split('morph')[1]), \
                       S.stats[stim]['nchoose1']/float(S.stats[stim]['ntrials'])) for stim in stimulus_list]

In [148]:
values

[('morph_0', 0.0),
 ('morph_1', 0.0),
 ('morph_3', 0.0),
 ('morph_5', 0.0),
 ('morph_6', 0.0),
 ('morph_7', 0.0),
 ('morph_8', 0.14285714285714285),
 ('morph_9', 0.14285714285714285),
 ('morph_12', 0.5714285714285714),
 ('morph_13', 0.875),
 ('morph_10', 0.375),
 ('morph_11', 0.5),
 ('morph_16', 1.0),
 ('morph_14', 0.625),
 ('morph_15', 0.7142857142857143),
 ('morph_2', 0.0),
 ('morph_4', 0.14285714285714285)]

#### Compare session numbers

In [15]:
dates = np.arange(all_sessions[0], all_sessions[-1])
print dates[0:10]

[20150202 20150203 20150204 20150205 20150206 20150207 20150208 20150209
 20150210 20150211]


In [ ]:
color_list = ['r', 'g', 'b']
pl.figure(figsize=(20,5))
for ai, animal in enumerate(sessions.keys()):
    session_list = sessions[animalid]
    pl.scatter([i for i in np.arange(len(dates))], [1+ai if dt in session_list else 0 for dt in dates], color=color_list[ai])
    

In [17]:
datestr = session_list[-10]
dfn = [f for f in raw_fns if animalid in f and str(datestr)[2:] in f] # saved datestr is YYMMDD.mwk
print dfn
dfn = dfn[0]
df = pymworks.open(dfn)

['/n/coxfs01/behavior-data/threeport_morphs/AG/raw/AG2_161130.mwk']


In [ ]:
codec = df.get_codec()

In [ ]:
codec

In [ ]:
df.get_events('TooFast_time')

In [ ]:
tstarts = [t for t in df.get_events('Announce_TrialStart') if t.value==1]
tends = [t for t in df.get_events('Announce_TrialEnd') if t.value==1]
print len(tstarts)
print len(tends)

In [ ]:
446+447+37